In [1]:
import pandas as pd
import os
from prophet import Prophet
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

C:\Users\82107\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### 데이터 불러오기

In [2]:
os.chdir('C:\\Users\\82107\\OneDrive\\바탕 화면\\KMU\\공부\\TP\\prophet\\simulation')

In [3]:
data = pd.read_csv('uniform_train_set.csv') 

### 날짜 형식 변경

In [4]:
# 날짜 형식 지정
date_format_in = '%d%b%Y:%H:%M:%S'
date_format_out = '%Y-%m-%d %H:%M:%S'

# 날짜 형식 변경
data['update_time'] = pd.to_datetime(data['update_time'], format=date_format_in)
data['update_time'] = data['update_time'].dt.strftime(date_format_out)

# datetime 형태로 변경
data['update_time'] = pd.to_datetime(data['update_time'])

In [5]:
data['update_time']

0      2020-01-01 00:00:00
1      2020-01-01 00:10:00
2      2020-01-01 00:20:00
3      2020-01-01 00:30:00
4      2020-01-01 00:40:00
               ...        
1345   2020-01-10 08:10:00
1346   2020-01-10 08:20:00
1347   2020-01-10 08:30:00
1348   2020-01-10 08:40:00
1349   2020-01-10 08:50:00
Name: update_time, Length: 1350, dtype: datetime64[ns]

In [6]:
data.shape

(1350, 2001)

In [7]:
data.head()

,heart_rate1,event1,heart_rate10,event10,heart_rate100,event100,heart_rate1000,event1000,heart_rate101,event101,...,event995,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,update_time
0,100,0,83,0,76,0,66,0,97,0,...,0,73,0,62,0,96,0,78,0,2020-01-01 00:00:00
1,79,0,91,0,77,0,83,0,96,0,...,0,88,0,96,0,96,0,91,0,2020-01-01 00:10:00
2,61,0,69,0,87,0,91,0,83,0,...,0,89,0,95,0,98,0,78,0,2020-01-01 00:20:00
3,71,0,66,0,89,0,89,0,93,0,...,0,94,0,75,0,100,0,88,0,2020-01-01 00:30:00
4,90,0,90,0,69,0,67,0,95,0,...,0,90,0,66,0,68,0,98,0,2020-01-01 00:40:00


In [8]:
# 최종 결과 데이터 만들기
pred = pd.DataFrame()

In [9]:
for i in range(1, 1001):
    # 필요한 변수만 추출
    df = data[['update_time', f'heart_rate{i}', f'event{i}', ]]
    # columns 명 변경
    df.rename(columns = {'update_time' : 'ds', f'heart_rate{i}' : 'y'}, inplace = True)
    
    # train 데이터와 test 데이터 나누기
    train1 = df[:1008]
    test = df[1008:]
    
    # 필요한 column만 추출
    train = train1[['ds', 'y']]
    test1 = test[['ds']]
    test1.reset_index(drop = True, inplace = True)
    test.reset_index(drop = True, inplace = True)
    
    # Prophet fit
    np.random.seed(i)
    model = Prophet(interval_width = 0.95)
    model.fit(train)
    
    # 예측
    forecast = model.predict(test1)
    
    # 결과
    performance = pd.merge(test, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on = 'ds')
    
    # 이상치 정의
    performance[f'pred_y{i}'] = performance.apply(lambda rows : 1 if ((rows.y < rows.yhat_lower)|(rows.y > rows.yhat_upper)) else 0, axis = 1)
    
    
    # 최종 데이터
    pred[f'heart_rate{i}'] = performance[['y']]
    pred[f'event{i}'] = test[[f'event{i}']]
    pred[f'pred_y{i}'] = performance[f'pred_y{i}']
    

23:33:02 - cmdstanpy - INFO - Chain [1] start processing
23:33:02 - cmdstanpy - INFO - Chain [1] done processing
23:33:03 - cmdstanpy - INFO - Chain [1] start processing
23:33:03 - cmdstanpy - INFO - Chain [1] done processing
23:33:04 - cmdstanpy - INFO - Chain [1] start processing
23:33:04 - cmdstanpy - INFO - Chain [1] done processing
23:33:05 - cmdstanpy - INFO - Chain [1] start processing
23:33:05 - cmdstanpy - INFO - Chain [1] done processing
23:33:06 - cmdstanpy - INFO - Chain [1] start processing
23:33:06 - cmdstanpy - INFO - Chain [1] done processing
23:33:06 - cmdstanpy - INFO - Chain [1] start processing
23:33:06 - cmdstanpy - INFO - Chain [1] done processing
23:33:07 - cmdstanpy - INFO - Chain [1] start processing
23:33:07 - cmdstanpy - INFO - Chain [1] done processing
23:33:08 - cmdstanpy - INFO - Chain [1] start processing
23:33:08 - cmdstanpy - INFO - Chain [1] done processing
23:33:09 - cmdstanpy - INFO - Chain [1] start processing
23:33:09 - cmdstanpy - INFO - Chain [1]

In [10]:
pred

,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,heart_rate4,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,81,0,0,87,0,0,88,0,0,81,...,0,66,0,0,92,0,0,82,0,0
1,100,0,0,92,0,0,87,0,0,71,...,0,73,0,0,92,0,0,85,0,0
2,73,0,0,73,0,0,99,0,0,88,...,0,87,0,0,62,0,0,92,0,0
3,95,0,0,98,0,0,69,0,0,65,...,0,86,0,0,83,0,0,94,0,0
4,91,0,0,68,0,0,100,0,0,98,...,0,136,1,1,84,0,0,72,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,109,1,1,73,0,0,72,0,0,95,...,0,88,0,0,137,1,1,100,0,0
338,71,0,0,85,0,0,132,1,1,106,...,0,93,0,0,108,1,1,71,0,0
339,83,0,0,88,0,0,74,0,0,77,...,0,87,0,0,81,0,0,87,0,0
340,91,0,0,73,0,0,72,0,0,60,...,0,96,0,0,73,0,0,76,0,0


In [11]:
test1.rename(columns = {'ds' : 'update_time'}, inplace = True)

In [12]:
predict = pd.concat([test1, pred], axis = 1)

In [13]:
predict

,update_time,heart_rate1,event1,pred_y1,heart_rate2,event2,pred_y2,heart_rate3,event3,pred_y3,...,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,heart_rate1000,event1000,pred_y1000
0,2020-01-08 00:00:00,81,0,0,87,0,0,88,0,0,...,0,66,0,0,92,0,0,82,0,0
1,2020-01-08 00:10:00,100,0,0,92,0,0,87,0,0,...,0,73,0,0,92,0,0,85,0,0
2,2020-01-08 00:20:00,73,0,0,73,0,0,99,0,0,...,0,87,0,0,62,0,0,92,0,0
3,2020-01-08 00:30:00,95,0,0,98,0,0,69,0,0,...,0,86,0,0,83,0,0,94,0,0
4,2020-01-08 00:40:00,91,0,0,68,0,0,100,0,0,...,0,136,1,1,84,0,0,72,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,2020-01-10 08:10:00,109,1,1,73,0,0,72,0,0,...,0,88,0,0,137,1,1,100,0,0
338,2020-01-10 08:20:00,71,0,0,85,0,0,132,1,1,...,0,93,0,0,108,1,1,71,0,0
339,2020-01-10 08:30:00,83,0,0,88,0,0,74,0,0,...,0,87,0,0,81,0,0,87,0,0
340,2020-01-10 08:40:00,91,0,0,73,0,0,72,0,0,...,0,96,0,0,73,0,0,76,0,0


In [16]:
predict.to_csv('all_uniform_Prophet_predict.csv')